## This notebook shows how to use Orchestrator APIs for user experiments

In [ ]:
import os
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json

In [ ]:
fabric_rc_location=os.environ['HOME']+"/work/fabric_config/fabric_rc"
if os.path.exists(fabric_rc_location):
    with open(fabric_rc_location, 'r') as f:
        for line in f:
            if line.startswith('export'):
                os.environ[line.split('=')[0].split('export')[1].strip()] = line.split('=')[1].strip()

In [ ]:
ssh_key_file_priv=os.environ["FABRIC_SLICE_PRIVATE_KEY_FILE"]
ssh_key_file_pub=os.environ["FABRIC_SLICE_PUBLIC_KEY_FILE"]

ssh_key_pub = None
with open (ssh_key_file_pub, "r") as myfile:
    ssh_key_pub=myfile.read()
    ssh_key_pub=ssh_key_pub.strip()

In [ ]:
slice_name="fabnet"

## Create Slice Manager Object
Users can request tokens with different Project and Scopes by altering `project_name` and `scope` parameters in the refresh call below.

In [ ]:
slice_manager = SliceManager()

### Orchestrator API example to query for available resources

In [ ]:
status, advertised_topology = slice_manager.resources()

print(f"Status: {status}")
if status == Status.OK:
    print(f"Toplogy: {advertised_topology}")
else:
    print(f"Error: {advertised_topology}")

In [ ]:
if status == Status.OK:
    advertised_topology.draw()

## Create Slice

In [ ]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, ComponentModelType, ServiceType
from fim.slivers.capacities_labels import Flags, Labels
from ipaddress import IPv4Network

# Create topology
t = ExperimentTopology()

# Set capacities
cap = Capacities(core=1, ram=4, disk=10)

site1 = "UKY2"
num_nodes = 2
name="node"
i=0
ifcs = []
# Auto Configure: Setup IP Addresses
network = IPv4Network("192.168.10.0/26")
ips = list(network.hosts())

for x in range(num_nodes):
    # Add node
    nm = f"{name}-{i}"
    
    n = t.add_node(name=f"{name}-{i}", site=site1)

    # Set properties
    n.set_properties(capacities=cap, image_type='qcow2', image_ref='default_rocky_8')
    #n.add_component(model_type=ComponentModelType.SharedNIC_ConnectX_6, name=f"{name}-{i}-nic1")
    n.add_component(model_type=ComponentModelType.SharedNIC_OpenStack_vNIC, name=f"{name}-{i}-nic1")
    
    # Auto Configure: Setup IP Addresses
    n.interface_list[0].flags = Flags(auto_config=True)
    n.interface_list[0].labels = Labels.update(n.interface_list[0].labels, ipv4=str(ips[i+1]))
    n.interface_list[0].labels = Labels.update(n.interface_list[0].labels, ipv4_subnet=str(network))
    print(n.interface_list[0].labels)
    ifcs.append(n.interface_list[0])
        
    i += 1

#t.add_network_service(name=f"fabnet-{site1}", nstype=ServiceType.FABNetv4, interfaces=ifcs)
ns = t.add_network_service(name='bridge1', nstype=ServiceType.L2Bridge, interfaces=ifcs)

# Generate Slice Graph
slice_graph = t.serialize()

# Request slice from Orchestrator
status, reservations = slice_manager.create(slice_name=slice_name, slice_graph=slice_graph, ssh_key=ssh_key_pub)


print("Response Status {}".format(status))
if status == Status.OK:
    print("Reservations created {}".format(reservations))
else:
    print(f"Failure: {reservations}")

## Query Slices

In [ ]:
slice_id = reservations[0].slice_id
status, slices = slice_manager.slices(slice_id=slice_id)


print("Response Status {}".format(status))
if status == Status.OK:
    slice_object = slices[0]
    print("Slice Name {}".format(slice_object.name))
    print("Slice State {}".format(slice_object.state))
    print("Slice End {}".format(slice_object.lease_end_time))
else:
    print(f"Failure: {slices}")

## Query Slivers

In [ ]:
status, slivers = slice_manager.slivers(slice_object=slice_object)

print("Response Status {}".format(status))
if status == Status.OK:
    for s in slivers:
        print("=====================")
        print("SliceId {}".format(s.slice_id))
        print("ReservationId {}".format(s.sliver_id))
        print("State         {}".format(s.state))
        print("Pending       {}".format(s.pending_state))
        print("Join State    {}".format(s.join_state))
        print("Sliver Type   {}".format(s.sliver_type))
        #print("Sliver        {}".format(s.sliver))
        print("Site        {}".format(s.fim_sliver.site))
        if s.sliver_type == "NodeSliver":
            print("Mgmt IP        {}".format(s.fim_sliver.management_ip))
        else:
            print("Subnet        {}".format(s.fim_sliver.gateway.subnet))
        
        if s.sliver_type == "NodeSliver":
            for d in s.fim_sliver.attached_components_info.devices.values():
                print("-------------------------")
                print("COMPONENT        {}".format(d))
                print("-------------------------")
        else:
            for d in s.fim_sliver.interface_info.interfaces.values():
                print("-------------------------")
                print("INTERFACE        {}".format(d))
                print("INTERFACE        {}".format(d.flags))
                print("-------------------------")
                
        print("Notices       {}".format(s.notice))
        
        print("=====================")
else:
    print(f"Failure: {slivers}")

## Get Slice Topology

In [ ]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, ComponentModelType, ServiceType
from fim.slivers.capacities_labels import Flags

status, existing_topology = slice_manager.get_slice_topology(slice_object=slice_object)
print("========EXISTING SLICE=======")
print(existing_topology)
print("========EXISTING SLICE=======")

## Modify Slice

### Add a Node to an existing Network Service

In [ ]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, ComponentModelType, ServiceType
from fim.slivers.capacities_labels import Flags

if status == Status.OK:
    name = "node"
    site1 = "UKY2"
    cap = Capacities(core=1, ram=4, disk=10)
    print(f"{name}-{i}")

    # Add a VM
    n3 = existing_topology.add_node(name=f"{name}-{i}", site=site1)
    n3.set_properties(capacities=cap, image_type='qcow2', image_ref='default_rocky_8')
    n3.add_component(model_type=ComponentModelType.SharedNIC_OpenStack_vNIC, name=f"{name}-{i}-nic4")
    n3.interface_list[0].flags = Flags(auto_config=True)
    n3.interface_list[0].labels = Labels.update(n3.interface_list[0].labels, ipv4=str(ips[i+1]))
    n3.interface_list[0].labels = Labels.update(n3.interface_list[0].labels, ipv4_subnet=str(network))
    
    
    existing_topology.network_services['bridge1'].disconnect_interface(interface=existing_topology.nodes['node-0'].interface_list[0])
    #existing_topology.remove_node(name='node-0')
    
    existing_topology.network_services['bridge1'].connect_interface(interface=n3.interface_list[0])

    print("========MODIFIED SLICE=======")
    print(existing_topology)
    existing_topology.validate()
    print("========MODIFIED SLICE=======")

### Remove L2Bridge and create FABNet instead and connect all nodes to it

In [ ]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, ComponentModelType, ServiceType
from fim.slivers.capacities_labels import Flags

if status == Status.OK:
    name = "node"
    site1 = "UKY"
    cap = Capacities(core=1, ram=4, disk=10)
    print(f"{name}-{i}")

    # Add a VM
    n3 = existing_topology.add_node(name=f"{name}-{i}", site=site1)
    n3.set_properties(capacities=cap, image_type='qcow2', image_ref='default_rocky_8')
    n3.add_component(model_type=ComponentModelType.SharedNIC_ConnectX_6, name=f"{name}-{i}-nic4")
    n3.interface_list[0].flags = Flags(auto_config=True)
    #n3.interface_list[0].labels = Labels.update(n3.interface_list[0].labels, ipv4=str(ips[i+1]))
    #n3.interface_list[0].labels = Labels.update(n3.interface_list[0].labels, ipv4_subnet=str(network))
    
    
    existing_topology.remove_network_service(name='bridge1')
    existing_topology.add_network_service(name='fabnet1', nstype=ServiceType.FABNetv4, interfaces=existing_topology.interface_list)

    print("========MODIFIED SLICE=======")
    print(existing_topology)
    existing_topology.validate()
    print("========MODIFIED SLICE=======")

In [ ]:
# Generate Slice Graph
slice_graph = existing_topology.serialize()

# Request slice from Orchestrator
status, reservations = slice_manager.modify(slice_id=slice_id, slice_graph=slice_graph)


print("Response Status {}".format(status))
if status == Status.OK:
    print("Reservations created {}".format(reservations))
else:
    print(f"Failure: {reservations}")

In [ ]:
status, slivers = slice_manager.slivers(slice_object=slice_object)

print("Response Status {}".format(status))
if status == Status.OK:
    for s in slivers:
        print("=====================")
        print("SliceId {}".format(s.slice_id))
        print("ReservationId {}".format(s.sliver_id))
        print("State         {}".format(s.state))
        print("Pending       {}".format(s.pending_state))
        print("Join State    {}".format(s.join_state))
        print("Sliver Type   {}".format(s.sliver_type))
        #print("Sliver        {}".format(s.sliver))
        print("Name        {}".format(s.fim_sliver.resource_name))
        print("Site        {}".format(s.fim_sliver.site))
        if s.sliver_type == "NodeSliver":
            print("Mgmt IP        {}".format(s.fim_sliver.management_ip))
        else:
            print("Subnet        {}".format(s.fim_sliver.gateway.subnet))
        
        if s.sliver_type == "NodeSliver":
            for d in s.fim_sliver.attached_components_info.devices.values():
                print("-------------------------")
                print("COMPONENT        {}".format(d))
                print("-------------------------")
        else:
            for d in s.fim_sliver.interface_info.interfaces.values():
                print("-------------------------")
                print("INTERFACE        {}".format(d))
                print("INTERFACE        {}".format(d.flags))
                print("-------------------------")
                
        print("Notices       {}".format(s.notice))
        
        print("=====================")
else:
    print(f"Failure: {slivers}")

In [ ]:
# Generate Slice Graph
# Request slice from Orchestrator
status, slice_topology = slice_manager.modify_accept(slice_id=slice_id)


print("Response Status {}".format(status))
if status == Status.OK:
    print(slice_topology)
else:
    print(f"Failure: {slice_topology}")

## Delete Slice

In [ ]:
status, result = slice_manager.delete(slice_object=slice_object)

print("Response Status {}".format(status))
print("Response received {}".format(result))